# Домашнее задание

1) Прочитать статьи про BPR, WARP loss

2) Сделать грид серч текущей модели

In [1]:
!pip3 install lightfm

     |████████████████████████████████| 310 kB 3.7 MB/s eta 0:00:01
  Created wheel for lightfm: filename=lightfm-1.16-cp39-cp39-macosx_10_9_x86_64.whl size=437174 sha256=7e3cd3684d895a2c9b9c7bf4136f07131e7f8bbc8663f338b1c027e8b946532b
  Stored in directory: /Users/ilya.ivolgin/Library/Caches/pip/wheels/d7/75/52/e42e5f9cd86d4902a352aff4dadde75ec041af713ffcf3ed05
Successfully built lightfm


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys, itertools

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/ilya.ivolgin/opt/miniconda3/envs/geek/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
def prefilter_items(data, take_n_popular=5000, item_features=None):
    # Уберем не интересные для рекоммендаций категории (department)
    if item_features is not None:
        department_size = pd.DataFrame(item_features. \
                                       groupby('department')['item_id'].nunique(). \
                                       sort_values(ascending=False)).reset_index()

        department_size.columns = ['department', 'n_items']
        rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
        items_in_rare_departments = item_features[
            item_features['department'].isin(rare_departments)].item_id.unique().tolist()

        data = data[~data['item_id'].isin(items_in_rare_departments)]

    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data = data[data['price'] > 2]

    # Уберем слишком дорогие товарыs
    data = data[data['price'] < 50]

    # Возбмем топ по популярности
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()

    # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999


    return data

In [4]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/var/folders/wf/0ws1q6ks6yv_xwm2_24s254h0000gn/T/ipykernel_20894/677787463.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


In [6]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [9]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [11]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [12]:
model = LightFM(no_components=40,
#                 loss='bpr',
                loss='warp',
                learning_rate=0.05, 
                item_alpha=0.1,
                user_alpha=0.1, 
                random_state=42)

In [13]:
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False) 

In [14]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [15]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [16]:
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision

0.051700678

In [17]:
test_item_ids = np.array([1, 2, 3, 200, 1200])

predictions = model.predict(user_ids=0,
                            item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)

In [18]:
test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

In [19]:
def grid_search(sparse_user_item, params, N=5):
    
    test_data = csr_matrix(test_user_item_matrix).tocsr()
    user_features = csr_matrix(user_feat_lightfm.values).tocsr()
    item_features = csr_matrix(item_feat_lightfm.values).tocsr()
    
    scores = []
    for components, loss, l_rate, alpha in itertools.product(*params.values()):
        model = LightFM(no_components=components,
                loss=loss,
                learning_rate=l_rate, 
                item_alpha=alpha,
                user_alpha=alpha, 
                random_state=42)
        
        model = model.fit((sparse_user_item > 0) * 1,
                           sample_weight=coo_matrix(user_item_matrix),
                           user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                           item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                           epochs=15, 
                           num_threads=6,
                           verbose=False) 
        
        precision = precision_at_k(model,
                                   test_data,
                                   user_features=user_features,
                                   item_features=item_features,
                                   k=N).mean()
                
        score = {'no_components': components,
                 'loss':          loss,
                 'learning_rate': l_rate,
                 'alpha':         alpha,
                 'precision':     precision}
        print(score)
        scores.append(score)
    
    return scores

def best_params(scores, all_=False):
    
    df = pd.DataFrame(scores, index=range(len(scores))).sort_values('precision', ascending=False)
    if all_:
        return df
    df = df.head(1)
    
    return df.to_dict()

In [20]:
%%time

parameters = {'no_components': [10, 100],
              'loss':          ['bpr', 'warp'],
              'learning_rate': [0.001, 0.1],
              'alpha':         [0.01, 0.1]}

scores = grid_search(sparse_user_item, parameters, N=5)

{'no_components': 10, 'loss': 'bpr', 'learning_rate': 0.001, 'alpha': 0.01, 'precision': 0.00019598237}
{'no_components': 10, 'loss': 'bpr', 'learning_rate': 0.001, 'alpha': 0.1, 'precision': 0.0}
{'no_components': 10, 'loss': 'bpr', 'learning_rate': 0.1, 'alpha': 0.01, 'precision': 0.0028417443}
{'no_components': 10, 'loss': 'bpr', 'learning_rate': 0.1, 'alpha': 0.1, 'precision': 9.7991186e-05}
{'no_components': 10, 'loss': 'warp', 'learning_rate': 0.001, 'alpha': 0.01, 'precision': 0.0008819207}
{'no_components': 10, 'loss': 'warp', 'learning_rate': 0.001, 'alpha': 0.1, 'precision': 0.00019598237}
{'no_components': 10, 'loss': 'warp', 'learning_rate': 0.1, 'alpha': 0.01, 'precision': 0.0027437531}
{'no_components': 10, 'loss': 'warp', 'learning_rate': 0.1, 'alpha': 0.1, 'precision': 0.005389515}
{'no_components': 100, 'loss': 'bpr', 'learning_rate': 0.001, 'alpha': 0.01, 'precision': 0.00019598237}
{'no_components': 100, 'loss': 'bpr', 'learning_rate': 0.001, 'alpha': 0.1, 'precision

In [21]:
best_params(scores, True)

,no_components,loss,learning_rate,alpha,precision
7,10,warp,0.100,0.10,0.005390
2,10,bpr,0.100,0.01,0.002842
10,100,bpr,0.100,0.01,0.002842
6,10,warp,0.100,0.01,0.002744
15,100,warp,0.100,0.10,0.002646
14,100,warp,0.100,0.01,0.002548
12,100,warp,0.001,0.01,0.001372
4,10,warp,0.001,0.01,0.000882
11,100,bpr,0.100,0.10,0.000784
9,100,bpr,0.001,0.10,0.000686


In [22]:
best_params(scores)

{'no_components': {7: 10},
 'loss': {7: 'warp'},
 'learning_rate': {7: 0.1},
 'alpha': {7: 0.1},
 'precision': {7: 0.005389514844864607}}